In [ ]:
#installing scrapy
!pip install scrapy

In [ ]:
#Importing the necessary packages
import numpy as np
import pandas as pd
import time 
import requests
from scrapy.http import TextResponse 

In [ ]:
url = "https://staff.am/en/jobs"
base_url = "https://staff.am"
page = requests.get(url)
response = TextResponse(url=page.url,body=page.text,encoding="utf-8")

In [ ]:
# Defining functions for companies, jobs, deadlines, locations, and individual pages
#which scrape the first page of the websited using css and xpath selectors interchangeably
#Deadline and location needed some data cleaning, which was perfomed by the codes inside their functions
def get_company(url):
  page = requests.get(url)
  response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
  company = response.css("p[class = 'job_list_company_title']::text").extract()
  return company 
  
def get_job(url):
  page = requests.get(url)
  response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
  job = response.css("p[class = 'font_bold']::text").extract()
  return job 

def get_deadline(url):
  page = requests.get(url)
  response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
  deadline = response.xpath("//div[@class='job-inner job-list-deadline']/p[1]/text()").extract()
  deadline = [i.replace("\n","") for i in deadline]
  deadline_updated = [i for i in deadline if len(i)>0]
  return deadline_updated

def get_location(url):
  page = requests.get(url)
  response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
  location = response.css("div[class = 'job-inner job-list-deadline'] > p[class = 'job_location']::text").extract()
  location = [i.replace("\n","") for i in location]
  location_updated = [i for i in location if len(i)>0]
  return location_updated

def get_individual_page(url):
  page = requests.get(url)
  response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
  individual_page = [base_url + i for i in response.css("div[class = 'jobListButtonsBlock job_list_button_block'] > div[class = 'job-inner-right text-right load-more-container pull-right'] > a[class = 'load-more btn width100 job_load_more radius_changes']::attr(href)").extract()]
  return individual_page


In [ ]:
# Definng all_pages with a range of 12 (number of pages on the website) to use it while scraping all pages
all_pages = ["https://staff.am/en/jobs?page={}&per-page=50".format(i) for i in range(1,13)]

In [ ]:
# Creating empty lists for each item and extending them with first scraped page to get the scraped info from all pages
all_companies = []
all_jobs = []
all_deadlines = []
all_locations = []
all_individual_pages = []
for i in all_pages:
    all_companies.extend(get_company(i))
    all_jobs.extend(get_job(i))
    all_deadlines.extend(get_deadline(i))
    all_locations.extend(get_location(i))
    all_individual_pages.extend(get_individual_page(i))
    time.sleep(1)

In [ ]:
# Combining empty lists and turning them into a DataFrame
df = pd.DataFrame(np.column_stack([all_companies, all_jobs, all_deadlines, all_locations, all_individual_pages]), columns = ['Companies', 'Jobs', 'Deadline', 'Location', 'Individual_Page'])

In [ ]:
# Saving the results in a csv file
 df.to_csv('jobs.csv', index=False) 

In [ ]:
# Reading the new csv file to perform analysis 
data = pd.read_csv("jobs.csv")

In [ ]:
# 1. The function value_counts() defines the number of job postings per company and head(5) shows the top 5
data.Companies.value_counts().head(5)

Digitain              32
SoftConstruct         29
PicsArt               23
ServiceTitan          16
TeamViewer Armenia    12
Name: Companies, dtype: int64

In [ ]:
# 2. As the word "data" is mostly used with a "D" in job postings, I needed to specify that it's lower case, 
# then look for all the results containing it in "Jobs" column, then count the values and sum them up to get the final number
data[data.Jobs.str.lower().str.contains("data")]["Jobs"].value_counts().sum()

10